In [25]:
import ipywidgets as widgets
import json
import time
import os
from IPython.display import display, HTML
from traitlets import Int, Unicode, List,observe

In [26]:
import requests
assert(requests.get('http://pshannon.systemsbiology.net/js/clustergrammer.js').status_code == 200)

In [16]:
# display(HTML('<link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.13/css/jquery.dataTables.min.css">'))
#    <link rel="stylesheet" href="css/custom.css" />
display(HTML('<link rel="stylesheet" type="text/css" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">'))
display(HTML('<link rel="stylesheet" type="text/css" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css")'))

In [22]:
display(HTML("<script src='http://cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min.js'></script>"))
display(HTML("<script src='http://pshannon.systemsbiology.net/js/clustergrammer.js'></script"))

In [17]:
#import ipywidgets as widgets
#from traitlets import Int, Unicode, List

class CircleWidget(widgets.DOMWidget):

    _view_name = Unicode('CircleView').tag(sync=True)
    _view_module = Unicode('circle').tag(sync=True)
    newCircleRequest = List().tag(sync=True)
    circleCount = Int(0).tag(sync=True);

    def drawCircle(self, x, y, radius):
       newCircle = {"x": x,  "y": y, "radius": radius};
       self.newCircleRequest = [newCircle];
        
    def getCount(self):
       return(self.circleCount);

In [23]:
%%javascript
"use strict";

require.config({

    paths: {'jquery'          : 'http://code.jquery.com/jquery-1.12.4.min',
            'jquery-ui'       : 'http://code.jquery.com/ui/1.12.1/jquery-ui.min',
            'jquery-dataTable': 'https://cdn.datatables.net/1.10.13/js/jquery.dataTables.min',
            'cytoscape'       : 'http://cytoscape.github.io/cytoscape.js/api/cytoscape.js-latest/cytoscape',
            'bootstrap'       : 'http://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min',
            'igv'             : 'http://igv.org/web/release/1.0.7/igv-1.0.7',
            'three'           : 'https://cdnjs.cloudflare.com/ajax/libs/three.js/r83/three',
            'underscore'      : 'https://cdnjs.cloudflare.com/ajax/libs/underscore.js/1.8.3/underscore-min',
            'd3'              : 'https://cdnjs.cloudflare.com/ajax/libs/d3/4.7.3/d3',
            'clustergrammer'  : 'http://pshannon.systemsbiology.net/js/clustergrammer',
            'app3d'           : 'http://pshannon.systemsbiology.net/js/app3d',
            },
    shim: {'bootstrap':   {'deps'   : ['jquery']},
           'igv':         {'deps'   : ['jquery', 'jquery-ui', 'bootstrap']},
           'three':       {'exports': 'THREE'},
           'underscore':  {'exports': '_'},
           }
    });


require.undef('circle');

define('circle', ["jupyter-js-widgets", "underscore", "jquery", "d3", "clustergrammer"], 
       function(widgets, _, $, d3, clustergrammer) {
    
    var CircleView = widgets.DOMWidgetView.extend({

        initialize: function() {
           console.log("--- CircleView.initialize")
           console.log("d3: ");
           console.log(d3);
           debugger;
           console.log("Clustergrammer: ");
           console.log(Clustergrammer);
            
           this.options = {};  // if this is missing: "Error setting state: view.options is undefined"
           this.circles = [];
           this.circleCount = 0;
           },

        createDiv: function(){
            var toolbarDiv = $("<div id='toolbarDiv' style='border:1px solid gray; height: 30px; width: 600px'></div>");
            var div = $("<div id='d3DemoDiv' style='border:1px solid red; height: 600px; width: 600px'></div>");
            var heatmapDiv = $("<div id='heatmapDiv' style='border:1px solid gray; height: 300px; width: 600px'><h3>heatmap</h3></div>");
            div.append(heatmapDiv);
            div.append(toolbarDiv);
            this.circleCountReadout = $("<input type='text' id='circleCountReadout' value='0'/>");
            toolbarDiv.append(this.circleCountReadout);
            var circleView = this;
            function clearCircles() {
               circleView.circles = [];
               circleView.circleCount = 0;
               $("#circleCountReadout").val(0);
               $("#svg").children().remove();
               circleView.model.set("circleCount", 0);
               circleView.touch();
               };

            var clearButton = $('<button>Clear Circles</button>').click(clearCircles);
            toolbarDiv.append(clearButton);
            return(div);
            },
  
        createCanvas: function(){
           var svg = d3.select("#d3DemoDiv")
                       .append("svg")
                       .attr("id", "svg").attr("width", 600).attr("height", 300);
           this.svg = svg;
           var circleView = this;
           svg.on('click', function() {
              var coords = d3.mouse(this);
              var newCircle = {x: coords[0], y: coords[1], radius: 20,
                               borderColor: "black", fillColor: "beige"};
              circleView.circles.push(newCircle);
              circleView.drawCircle(newCircle, "blue");
              });
           }, 

        createHeatmap: function() {
          var network_data =
            {"views":
               [{"N_row_sum": "all", "dist": "cos", "nodes":
                  {"col_nodes": [{"ini": 3, "rank": 2, "name": "s01_120405", "clust": 1,
                                  "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 2},
                                 {"ini": 2, "rank": 1, "name": "s02_120405", "clust": 0,
                                  "group": [3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                                 {"ini": 1, "rank": 0, "name": "s03_120405", "clust": 2,
                                  "group": [2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 1}],
                   "row_nodes": [{"ini": 2, "rank": 0, "name": "HLTF", "clust": 0,
                                  "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                                 {"ini": 1, "rank": 1, "name": "POU2F1", "clust": 1,
                                  "group": [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0], "rankvar": 1}]}},
                {"dist": "cos", "N_row_var": "all", "nodes":
                  {"col_nodes": [{"ini": 3, "rank": 2, "name": "s01_120405", "clust": 1,
                                  "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 2},
                                 {"ini": 2, "rank": 1, "name": "s02_120405", "clust": 0,
                                  "group": [3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                                 {"ini": 1, "rank": 0, "name": "s03_120405", "clust": 2,
                                  "group": [2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 1}],
                   "row_nodes": [{"ini": 2, "rank": 0, "name": "HLTF", "clust": 0,
                                  "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                                 {"ini": 1, "rank": 1, "name": "POU2F1", "clust": 1,
                                  "group": [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0], "rankvar": 1}]}}],
               "col_nodes": [{"ini": 3, "rank": 2, "name": "s01_120405", "clust": 1,
                              "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 2},
                             {"ini": 2, "rank": 1, "name": "s02_120405", "clust": 0,
                              "group": [3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                             {"ini": 1, "rank": 0, "name": "s03_120405", "clust": 2,
                              "group": [2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 1}],
               "links": [{"target": 0, "value": 5.591991, "source": 0},
                         {"target": 1, "value": 11.939007, "source": 0},
                         {"target": 2, "value": 7.738552, "source": 0},
                         {"target": 0, "value": 31.060965999999997, "source": 1},
                         {"target": 1, "value": 18.00348, "source": 1},
                         {"target": 2, "value": 21.577569, "source": 1}],
               "row_nodes": [{"ini": 2, "rank": 0, "name": "HLTF", "clust": 0,
                              "group": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], "rankvar": 0},
                             {"ini": 1, "rank": 1, "name": "POU2F1", "clust": 1,
                              "group": [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0], "rankvar": 1}]};
       
          make_clust(network_data);
          var about_string = 'Zoom, scroll, and click buttons to interact with the clustergram. <a href="http://amp.pharm.mssm.edu/clustergrammer/help"> <i class="fa fa-question-circle" aria-hidden="true"></i> </a>';
          function make_clust(network_data){
             var args = {
                'root': '#heatmapDiv',
                'network_data': network_data,
                'about': about_string,
                'sidebar_width':150,
                };
             var screen_width = window.innerWidth;
             var screen_height = 300;
             $("#heatmapDiv").width(screen_width);
             $("#heatmapDiv").height(screen_height);
             cgm = Clustergrammer(args);
             $("#heatmapDiv .wait_message").remove()
             console.log('loading clustergrammer')
             } // make_clust
         }, // createHeatmap

        drawCircle: function(obj, color){
           this.svg.append("circle")
              .style("stroke", color)
              .style("fill", "white")
              .attr("r", obj.radius)
              .attr("cx", obj.x)
              .attr("cy", obj.y)
              .on("mouseover", function(){d3.select(this).style("fill", "aliceblue");})
              .on("mouseout",  function(){d3.select(this).style("fill", "white");});
           this.circleCount += 1;
           $("#circleCountReadout").val(this.circleCount);
           this.model.set("circleCount", this.circleCount);
           this.touch();
           },

        render: function() { 
            this.$el.append(this.createDiv());
            this.listenTo(this.model, 'change:newCircleRequest', this.newCircleRequested, this);
            var circleView = this;
            function delayCanvasCreationUntilDivExists(){ 
                 // would be better to trigger on DOM div creation. not knowing how to do that, 
                 // this setTimeout hack will have to suffice.
                circleView.createCanvas();
                circleView.createHeatmap();
               }
            setTimeout(delayCanvasCreationUntilDivExists, 0);
            },

        newCircleRequested: function() {
           var newCircle = this.model.get("newCircleRequest")[0];
           this.circles.push(newCircle);
           this.drawCircle(newCircle, "red");
           }

    });
    return {
        CircleView : CircleView
    };
});

<IPython.core.display.Javascript object>

In [24]:
cw = CircleWidget(width=500, height=300)
cw

In [10]:
cw.drawCircle(x=30, y=30, radius=10)

In [13]:
cw.drawCircle(x=400, y=200, radius=30)

In [ ]:
cw.getCount()